<a href="https://colab.research.google.com/github/mhask94/cs474_labs_f2019/blob/conv2linear/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# imports for training data
!if ( ! ls . | grep pytransform ); then git clone https://github.com/mhask94/pytransform.git; fi
# !git clone https://github.com/mhask94/pytransform.git
from pytransform.common import skew
from pytransform.quaternion import Quaternion as Quat
import numpy as np

Cloning into 'pytransform'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 65 (delta 6), reused 14 (delta 6), pack-reused 51
Unpacking objects: 100% (65/65), done.


In [0]:
# imports for pytorch
import torch
import torch.nn as nn


In [0]:
# classes for quadrotor state and dynamics
class State():
    def __init__(self, arr=np.empty(0)):
        if len(arr) == 0:
            self.arr = np.zeros((10,1), dtype=np.float64)
            self.arr[3] = 1
        else:
            assert arr.shape == (10, 1)
            if not arr.dtype == np.float64:
              arr = np.array(arr, dtype=np.float64)
            arr.dtype = np.float64
            self.arr = arr

    def __getitem__(self, position):
        return self.arr[position]
    def __str__(self):
        s = 'p: ' + str(self.p.flatten()) + '\nq: ' + self.q.__str__() + \
                '\nv: ' + str(self.v.flatten())
        s = s.replace('[ ', '[')
        s = s.replace(', ', ' ')
        s = s.replace(' ]', ']')
        return s
    def __repr__(self):
        return self.__str__()
    def __add__(self, other):
        assert other.shape == (9, 1)
        out = np.empty(self.arr.shape)
        out[:3]  = self.p + other[:3]
        out[3:7] = (self.q + other[3:6]).elements
        out[7:]  = self.v + other[6:]
        return State(out)
    def __iadd__(self, other):
        assert other.shape == (9, 1)
        self.arr[:3] += other[:3]
        self.arr[3:7] = (self.q + other[3:6]).elements
        self.arr[7:] += other[6:]
        return self
    @property
    def p(self):
        return self.arr[:3]
    @property
    def q(self):
        return Quat(self.arr[3:7])
    @property
    def v(self):
        return self.arr[7:]
    @property
    def elements(self):
        return self.arr.copy()
    def copy(self):
        return State(self.arr.copy())

class Dynamics():
    def __init__(self):
        self.k1 = np.zeros((9,1))
        self.k2 = np.zeros((9,1))
        self.k3 = np.zeros((9,1))
        self.k4 = np.zeros((9,1))
        self.cd = 0.1
        e_z = np.array([[0,0,1]]).T
        self.g = 9.8065 * e_z
        self.se = 0.5

    def run(self, xu, dt):
        x,u = State(xu[:10]), xu[10:]
        self.k1 = self.f(x, u)
        self.k2 = self.f(x + self.k1*(dt/2), u)
        self.k3 = self.f(x + self.k2*(dt/2), u)
        self.k4 = self.f(x + self.k3*dt, u)
        # x += (self.k1 + 2*(self.k2 + self.k3) + self.k4) * (dt/6)
        return x + (self.k1 + 2*(self.k2 + self.k3) + self.k4) * (dt/6)

    def f(self, x, u):
        s, w = u[0], u[1:]
        dx = np.empty(self.k1.shape)
        dx[:3] = x.q.rota(x.v)
        dx[3:6] = w
        dx[6:] = -self.g*(s/self.se) - self.cd*x.v + x.q.rotp(self.g) - \
                skew(w) @ x.v
        return dx

    @property
    def state(self):
        return self.x.copy()

In [0]:
class DataGenerator():
  def __init__(self, num_states, num_inputs, batch_size=50):
    self.n = num_states
    self.m = num_inputs
    self.batch_size = batch_size
    self.pos_lim = 300
    self.att_lim = np.pi
    self.vel_lim = 10
    self.rate_lim = 2*np.pi
    self.s_lim = 1
    self.dyn = Dynamics()

  def getRandomInput(self):
    xu = np.empty(self.n + self.m)
    xu[:2] = np.random.uniform(-self.pos_lim, self.pos_lim, 2)
    xu[2] = np.random.uniform(-self.pos_lim, 0)
    mask = np.random.uniform(size=3) > 0.2
    euler = np.random.uniform(-self.att_lim, self.att_lim, 3) * mask
    xu[3:7] = Quat.from_euler(*euler).elements.flatten()
    xu[7:10] = np.random.uniform(-self.vel_lim, self.vel_lim, 3)
    xu[10] = np.random.uniform(0, self.s_lim)
    xu[11:] = np.random.uniform(-self.rate_lim, self.rate_lim, 3)
    return xu

  def getBatch(self):
    batch_in = np.empty((self.batch_size, self.n+self.m))
    batch_out = np.empty((self.batch_size, self.n))
    for i in range(self.batch_size):
      batch_in[i] = self.getRandomInput()
      # set_trace()
      batch_out[i] = self.dyn.run(batch_in[i].reshape(-1,1), 0.01).elements.flatten()
    return batch_in, batch_out

In [7]:
def testGen():
  data_gen = DataGenerator(10,4,batch_size=2)
  ran = data_gen.getRandomInput()
  print('rand: ', ran)

  x, truth = data_gen.getBatch()
  print('x: \n', x)
  print('truth: \n', truth)

  dyn = Dynamics()
  for i, state in enumerate(x):
    state = state.reshape(-1,1)
    out = dyn.run(state, 0.01).elements.flatten()
    error = out - truth[i]
    norm = np.sqrt(error @ error)
    print('norm: ', norm)

testGen()

rand:  [-2.37376877e+02 -2.33592496e+02 -2.40692200e+02  1.82575767e-01
 -7.12876906e-01  6.72179964e-01 -8.15273053e-02 -9.93346121e+00
 -1.79908616e-01 -9.00031686e+00  2.28967991e-01 -4.78940293e+00
  2.44998240e+00  3.26534699e+00]
x: 
 [[-2.99883248e+02 -2.64677063e+02 -2.33598845e+02  6.96834736e-01
  -1.28217208e-01  6.93678477e-01  1.29583443e-01 -3.45969480e+00
  -6.42226660e+00  8.51704652e+00  3.73301944e-01 -4.22368776e+00
  -3.61069116e+00 -5.75473210e+00]
 [ 2.12280722e+02 -1.26793447e+02 -2.42394608e+02  8.78024204e-01
   2.93601413e-01 -3.04814016e-01 -2.23517614e-01  6.92765883e+00
  -2.85759238e+00  4.79175559e+00  7.68715272e-01  3.96231051e+00
   4.73566946e+00  5.02678594e+00]]
truth: 
 [[-2.99781247e+02 -2.64706702e+02 -2.33563447e+02  7.09817924e-01
  -1.60442339e-01  6.74122578e-01  1.26394308e-01 -2.86623677e+00
  -6.95508759e+00  8.26925267e+00]
 [ 2.12292654e+02 -1.26871669e+02 -2.42354506e+02  8.84345330e-01
   3.08391190e-01 -2.95592301e-01 -1.88290740e-01 

In [8]:
def testLinearSize():
  inputs = 14
  outputs = 10
  batch = 1
  x_test = torch.zeros(batch,inputs)
  up = nn.Linear(inputs, 50)
  up_test = up(x_test)
  print('up: ', up_test.size())

  down = nn.Linear(50, outputs)
  down_test = down(up_test)
  print('down: ', down_test.size())

testLinearSize()

up:  torch.Size([1, 50])
down:  torch.Size([1, 10])


In [0]:
from IPython.core.debugger import set_trace
class ResBlock(nn.Module):
  def __init__(self, dim_in, dim_out, skip=False):
    super(ResBlock, self).__init__()
    self.skip = skip
    self.activation = nn.ReLU()
    self.layer1 = nn.Linear(dim_in, dim_in)
    self.layer2 = nn.Linear(dim_in, dim_in)
    self.layer3 = nn.Linear(dim_in, dim_out)

  def forward(self, x):
    out1 = self.activation(self.layer1(x))
    out2 = self.layer2(out1)
    skip = self.activation(out2 + out1)
    out3 = self.activation(self.layer3(skip))
    if self.skip:
      return skip, out3
    else:
      return out3

class DynamicsNN(nn.Module):
  def __init__(self, num_states, num_inputs):
    super(DynamicsNN, self).__init__()
    self.up1 = nn.Linear(num_states+num_inputs, 50)
    self.up2 = ResBlock(50,  100, skip=True)
    self.up3 = ResBlock(100, 200, skip=True)
    # self.up4 = ResBlock(200, 400, skip=True)
    self.dn1 = ResBlock(200, 100)
    self.dn2 = ResBlock(100, 50)
    self.dn3 = ResBlock(50,  num_states)
    # self.dn4 = ResBlock(50, num_states)

  def forward(self, x):
    # set_trace()
    up1 = self.up1(x)
    skip1, up2 = self.up2(up1)
    skip2, up3 = self.up3(up2)
    down1 = self.dn1(up3)
    down2 = self.dn2(skip2 + down1)
    out = self.dn3(skip1 + down2)
    out[:,3:7] /= torch.norm(out[:,3:7], dim=1)
    mask = (out[:,3] < 0).squeeze()
    out[mask,3:7] *= -1
    return out

In [20]:
def testNet():
  n = 10
  m = 4
  x_test = torch.randn(1,n+m)
  print('x_test: ', x_test)
  net = DynamicsNN(n,m)
  test = net(x_test)
  print('shape: ', test.shape)
  print('output: ', test)

testNet()

x_test:  tensor([[-1.5526, -0.5519, -0.3450, -2.1592, -1.4334, -0.5845, -1.5252, -0.2106,
         -1.3286, -0.5176,  0.7229,  0.2374, -0.5309,  0.0791]])
shape:  torch.Size([1, 10])
output:  tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.4420, 0.0000, 0.8970, 0.0727, 0.0386,
         0.0000]], grad_fn=<CopySlices>)
